In [1]:
AGENT_NAME = r'20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
#SAVE_DIR = 'PPO agent 100 alts over 1000+200 2-3-21 results' +'/'

In [2]:
from stable_baselines3 import PPO
from citylearn.data import DataSet
import KBMproject.utilities as utils
import pandas as pd

In [3]:
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True,)
print('Model loaded from storage')
bins = agent.action_space[0].n

schema = DataSet.get_schema(DATASET_NAME)
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

Model loaded from storage


In [4]:
cols = env.observation_names

In [5]:
baseline_kpis, baseline_obs, baseline_a = utils.eval_agent(env,agent)
display(baseline_kpis)

cost_function
annual_peak_average                      1.102271
carbon_emissions_total                   0.869186
cost_total                               0.787803
daily_one_minus_load_factor_average      1.095762
daily_peak_average                       0.894595
electricity_consumption_total            0.879108
monthly_one_minus_load_factor_average    0.988364
ramping_average                          1.093749
zero_net_energy                          1.107710
Name: District, dtype: float64

In [6]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename,
                          index_col=0)
    df_kpis['baseline'] = baseline_kpis.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    baseline_kpis.name = 'baseline'
    baseline_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/KPIs.csv created


In [7]:
df_sa = pd.DataFrame(baseline_obs)
df_sa.columns = cols
df_sa['actions'] = baseline_a

In [8]:
df_sa.to_csv(SAVE_DIR+'baseline_obs-a.csv')